In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 이상치를 찾아라

### 1. 데이터에서 IQR을 활용해 Fare컬럼의 이상치를 찾고, 이상치 데이터의 여성 수를 구하시오
- 데이터셋 : Titanic.csv

In [ ]:
import pandas as pd
kaggle_path = '/kaggle/input/bigdatacertificationkr'
titanic = pd.read_csv(f'{kaggle_path}/Titanic.csv')
titanic.head()

In [ ]:
titanic['Fare'].describe()

In [ ]:
import numpy as np
q1,q3 = titanic['Fare'].quantile(.25),titanic['Fare'].quantile(.75)
iqr = titanic['Fare'].quantile(.75) - titanic['Fare'].quantile(.25)
min_iqr,max_iqr = q1-(1.5*iqr),q3+(1.5*iqr)

np.where(titanic['Fare'] > max_iqr)
ex1 = titanic.iloc[np.where(titanic['Fare'] > max_iqr)]
len(ex1[ex1['Gender'] == 'female'])

### 2. 주어진 데이터에서 이상치(소수점 나이)를 찾고 올림, 내림, 버림(절사)했을때 3가지 모두 이상치 'age' 평균을 구한 다음 모두 더하여 출력하시오
- 데이터셋 : basic1.csv

In [ ]:
basic1 = pd.read_csv(f'{kaggle_path}/basic1.csv')
basic1.head()

In [ ]:
basic1['age'].describe()

In [ ]:
new = basic1[basic1['age'] - np.floor(basic1['age']) != 0]
np.ceil(new['age']).mean() + np.floor(new['age']).mean() + np.trunc(new['age']).mean()

# 결측치 처리
### 1. 주어진 데이터에서 결측치가 80%이상 되는 컬럼은(변수는) 삭제하고, 80% 미만인 결측치가 있는 컬럼은 'city'별 중앙값으로 값을 대체하고 'f1'컬럼의 평균값을 출력하세요!
- 데이터셋 : basic1.csv

In [ ]:
basic1 = pd.read_csv(f'{kaggle_path}/basic1.csv')
basic1.head()

In [ ]:
basic1.info()

In [ ]:
temp_dict = dict()
for city in basic1['city'].unique():
    if city not in temp_dict.keys():
        medi = basic1[basic1['city'] == city]['f1'].median()
        temp_dict[city] = medi
basic1['f1'] = basic1['f1'].fillna(basic1['city'].map(temp_dict))
basic1 = basic1.drop(['f3'],axis=1)
basic1['f1'].mean()
# basic1['f1'].fillna()
# help(pd.DataFrame.fillna)

# 왜도"skew"와 첨도"kurt"
### 1. 주어진 데이터 중 train.csv에서 'SalePrice'컬럼의 왜도와 첨도를 구한 값과, 'SalePrice'컬럼을 스케일링(log1p)로 변환한 이후 왜도와 첨도를 구해 모두 더한 다음 소수점 2째자리까지 출력하시오
    - 데이터셋 : /kaggle/input/house-prices-advanced-regression-techniques/train.csv

In [ ]:
house = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
house.head()

In [ ]:
log_sale = np.log1p(house['SalePrice'])
round(log_sale.skew()+log_sale.kurt(),2)

In [ ]:
# dir(pd.DataFrame)
help(round)

# 조건에 맞는 데이터 표준편차 구하기
### 주어진 데이터 중 basic1.csv에서 'f4'컬럼 값이 'ENFJ'와 'INFP'인 'f1'의 표준편차 차이를 절대값으로 구하시오
    - 데이터셋 : basic1.csv

In [ ]:
basic1 = pd.read_csv('/kaggle/input/bigdatacertificationkr/basic1.csv')
basic1.head()

In [ ]:
enfj = basic1[basic1['f4'] == 'ENFJ']
infp = basic1[basic1['f4'] == 'INFP']
abs(enfj['f1'].std() - infp['f1'].std())

# 결측치 제거 및 그룹 합계에서 조건에 맞는 값 찾아 출력
### 주어진 데이터 중 basic1.csv에서 'f1'컬럼 결측 데이터를 제거하고, 'city'와 'f2'을 기준으로 묶어 합계를 구하고, 'city가 경기이면서 f2가 0'인 조건에 만족하는 f1 값을 구하시오
    - 데이터셋 : basic1.csv

In [ ]:
basic1 = pd.read_csv('/kaggle/input/bigdatacertificationkr/basic1.csv')
# new = basic1.drop(basic1[basic1['f1'].isna()].index)
new = basic1[~basic1['f1'].isnull()]
new.groupby(['city','f2'])['f1'].sum()[0]

In [ ]:
# help(pd.DataFrame.groupby)

### 'f4'컬럼의 값이 'ESFJ'인 데이터를 'ISFJ'로 대체하고, 'city'가 '경기'이면서 'f4'가 'ISFJ'인 데이터 중 'age'컬럼의 최대값을 출력하시오!
    - 데이터셋 : basic1.csv

In [ ]:
basic1 = pd.read_csv('/kaggle/input/bigdatacertificationkr/basic1.csv')
basic1.head()
# print(basic1['f4'].value_counts())
basic1['f4'] = basic1['f4'].replace('ESFJ','ISFJ')
# print(basic1['f4'].value_counts())
max(basic1[(basic1['city'] == '경기') & (basic1['f4'] == 'ISFJ')]['age'])

In [ ]:
# help(pd.DataFrame.replace)

# 누적합 그리고 보간(결측치 처리)

### 주어진 데이터 셋에서 'f2' 컬럼이 1인 조건에 해당하는 데이터의 'f1'컬럼 누적합을 계산한다. 이때 발생하는 누적합 결측치는 바로 뒤의 값을 채우고, 누적합의 평균값을 출력한다. (단, 결측치 바로 뒤의 값이 없으면 다음에 나오는 값을 채워넣는다)
    - 데이터셋 : basic1.csv


In [ ]:
basic1 = pd.read_csv('/kaggle/input/bigdatacertificationkr/basic1.csv')
new = basic1[basic1['f2']== 1]
(new['f1'].fillna(method='backfill').cumsum()).mean(),(new['f1'].fillna(method='backfill').sum()).mean()

In [ ]:
# help(pd.DataFrame.fillna)
for i in dir(pd.DataFrame):
    if 'sum' in i:
        print(i)

# 수치형 변수 변환하기

### 1. 주어진 데이터에서 'f5'컬럼을 표준화(Standardization (Z-score Normalization))하고 그 중앙값을 구하시오
    - 데이터셋 : basic1.csv


In [ ]:
basic1 = pd.read_csv('/kaggle/input/bigdatacertificationkr/basic1.csv')
# basic1['f5'].info()
import sklearn
from sklearn import *

stdscaler = sklearn.preprocessing.StandardScaler()
np.median(stdscaler.fit_transform(basic1[['f5']]))

In [ ]:
# print(sklearn.__all__)
# help(sklearn.preprocessing.StandardScaler)

### 2. 주어진 데이터에서 20세 이상인 데이터를 추출하고 'f1'컬럼을 결측치를 최빈값으로 채운 후, f1 컬럼의 여-존슨과 박스콕스 변환 값을 구하고, 두 값의 차이를 절대값으로 구한다음 모두 더해 소수점 둘째 자리까지 출력(반올림)하시오
    - 데이터셋 : basic1.csv


In [ ]:
basic1 = pd.read_csv('/kaggle/input/bigdatacertificationkr/basic1.csv')
basic1.head()
new = basic1[basic1['age'] >= 20]
new['f1'] = new['f1'].fillna(new['f1'].mode()[0])

import sklearn
from sklearn import *

print(sklearn.preprocessing.__all__)
np.round(sum(abs(sklearn.preprocessing.power_transform(new[['f1']],standardize=False) - sklearn.preprocessing.power_transform(new[['f1']],method='box-cox',standardize=False))),2)[0]
# help(pd.DataFrame.fillna)

# min-max 기준 상하위 5%값
### 주어진 데이터에서 'f5'컬럼을 min-max 스케일 변환한 후, 상위 5%와 하위 5% 값의 합을 구하시오
    - 데이터셋 : basic1.csv

In [ ]:
basic1 = pd.read_csv('/kaggle/input/bigdatacertificationkr/basic1.csv')

from sklearn.preprocessing import MinMaxScaler
import numpy as np 
minmax = MinMaxScaler()
basic1['f5'] = minmax.fit_transform(basic1[['f5']])
np.percentile(minmax_f5,95) + np.percentile(minmax_f5,5)
# basic1['f5'].quantile(.95)

In [ ]:
# help(np.percentile)

# 상위 10개, 하위 10개 차이

### 주어진 데이터에서 상위 10개 국가의 접종률 평균과 하위 10개 국가의 접종률 평균을 구하고, 그 차이를 구해보세요 (단, 100%가 넘는 접종률 제거, 소수 첫째자리까지 출력)

    - 데이터셋 : ../input/covid-vaccination-vs-death/covid-vaccination-vs-death_ratio.csv

In [ ]:
import pandas as pd
import numpy as np
covid = pd.read_csv('/kaggle/input/covid-vaccination-vs-death/covid-vaccination-vs-death_ratio.csv')
covid.head()

In [ ]:
new = covid[['country','ratio']]
new = new[new['ratio'] <= 100]
new['ratio'] = round(new['ratio'],1)

# print(new.groupby('country').max()) 기간별 row라서 국가별 최종 접종률만 필요
print((new.groupby('country').max().sort_values(by='ratio',ascending=False).head(10).mean() - new.groupby('country').max().sort_values(by='ratio',ascending=False).tail(10).mean())[0])

In [ ]:
# help(pd.DataFrame.loc)

# 3유형 대응(쌍체)표본 t검정
    - 귀무가설(H0): μ >= 0
    - 대립가설(H1): μ < 0
    - μ = (치료 후 혈압 - 치료 전 혈압)의 평균
    - 유의수준: 0.05
1. μ의 표본평균은?(소수 둘째자리까지 반올림)
2. 검정통계량 값은?(소수 넷째자리까지 반올림)
3. p-값은?(소수 넷째자리까지 반올림)
4. 가설검정의 결과는? (유의수준 5%)

In [ ]:
import pandas as pd
high_blood = pd.read_csv('/kaggle/input/bigdatacertificationkr/high_blood_pressure.csv')
high_blood['u'] = high_blood['bp_post'] - high_blood['bp_pre']
print(f"표본 평균 {round((high_blood['u']).mean(),2)}")

import scipy
from scipy import *

# help(scipy.stats.ttest_rel)
print(f"검정통계량 {round(scipy.stats.ttest_rel(high_blood['bp_post'],high_blood['bp_pre'],alternative='less').statistic,4)}")
print(f"p-value {round(scipy.stats.ttest_rel(high_blood['bp_post'],high_blood['bp_pre'],alternative='less').pvalue,4)}")
print("유의 수준 0.025보다 작으므로 귀무가설을 기각하고 대립가설을 채택 ")
#TtestResult(statistic=-3.0002163948827545, pvalue=0.0032868948457022056, df=119)

# 3유형 독립t검정
### Q: 어떤 특정 약물을 복용한 사람들의 평균 체온이 복용하지 않은 사람들의 평균 체온과 유의미하게 다른지 검정해보려고 합니다.
가정:약물을 복용한 그룹과 복용하지 않은 그룹의 체온 데이터가 각각 주어져 있다고 가정합니다.
각 그룹의 체온은 정규분포를 따른다고 가정합니다.  

**검정통계량, p-value, 검정결과를 출력하시오**


In [ ]:
# 데이터 수집
group1 = [36.8, 36.7, 37.1, 36.9, 37.2, 36.8, 36.9, 37.1, 36.7, 37.1]
group2 = [36.5, 36.6, 36.3, 36.6, 36.9, 36.7, 36.7, 36.8, 36.5, 36.7]

import scipy
from scipy import stats
# for i in dir(scipy.stats):
#     if 'test' in i:
#         print(i)
# help(scipy.stats.ttest_ind)
statistic,pvalue = scipy.stats.ttest_ind(group1,group2)
print(f'검정통계량 {statistic}')
print(f'p-val {pvalue}')
print('대립가설 채택')

# 3유형 단일t검정

### 문제: 다음은 22명의 학생들이 국어시험에서 받은 점수이다. 학생들의 평균이 75보다 크다고 할 수 있는가?
- 귀무가설(H0): 모평균은 mu와 같다. (μ = mu), 학생들의 평균은 75이다
- 대립가설(H1): 모평균은 mu보다 크다. (μ > mu), 학생들의 평균은 75보다 크다  
가정:
- 모집단은 정규분포를 따른다.
- 표본의 크기가 충분히 크다.  

** 검정통계량, p-value, 검정결과를 출력하시오 **



In [ ]:
import scipy
from scipy import *
# data
scores = [75, 80, 68, 72, 77, 82, 81, 79, 70, 74, 76, 78, 81, 73, 81, 78, 75, 72, 74, 79, 78, 79]
# for i in dir(scipy.stats):
#     if 'ttest' in i:
#         print(i)
        
statistic,pval = scipy.stats.ttest_1samp(scores,75,alternative='greater')
print(statistic)
print(pval)
print('대립가설 채택')

# 3유형 ANOVA
### 세 가지 다른 교육 방법(A, B, C)을 사용하여 수험생들의 시험 성적을 개선시키는 효과를 평가하고자 한다. 30명의 학생들을 무작위로 세 그룹으로 배정하여 교육을 실시하였고, 시험을 보고 성적을 측정하였습니다. 다음은 각 그룹의 학생들의 성적 데이터입니다.
    - 귀무가설(H0): 세 그룹(A, B, C) 간의 평균 성적 차이가 없다.
    - 대립가설(H1 또는 Ha): 세 그룹(A, B, C) 간의 평균 성적 차이가 있다.  
#### 일원배치법을 수행하여 그룹 간의 평균 성적 차이가 있는지 검정하세요
    1. f값 (소수 둘째자리)
    2. p값 (소수 여섯째자리)
    3. 검정결과 출력

In [ ]:
# 각 그룹의 데이터
groupA = [85, 92, 78, 88, 83, 90, 76, 84, 92, 87]
groupB = [79, 69, 84, 78, 79, 83, 79, 81, 86, 88]
groupC = [75, 68, 74, 65, 77, 72, 70, 73, 78, 75]

import scipy.stats
# help(scipy.stats.f_oneway)
f_val,t_val = stats.f_oneway(groupA,groupB,groupC)
print(round(f_val,2))
print(format(t_val,'.6f'))
print('H1')

# 3유형 shapiro-wilk 검정
### 12명의 수험생이 빅데이터 분석기사 시험에서 받은 점수이다. Shapiro-Wilk 검정을 사용하여 데이터가 정규 분포를 따르는지 검증하시오
    - 귀무 가설(H0): 데이터는 정규 분포를 따른다.
    - 대립 가설(H1): 데이터는 정규 분포를 따르지 않는다.
**Shapiro-Wilk 검정 통계량, p-value, 검증결과를 출력하시오**


In [5]:
import scipy.stats
data = [75, 83, 81, 92, 68, 77, 78, 80, 85, 95, 79, 89]

# for i in dir(scipy.stats):
#     if 'sha' in i:
#         print(i)
        
# help(scipy.stats.shapiro)
statistic,pval = scipy.stats.shapiro(data)
print(statistic)
print(pval)
print('귀무 가설 기각 불가')

0.9768089652061462
0.9676492810249329
귀무 가설 기각 불가


# 3유형 Correlation
### iris에서 Sepal Length와 Sepal Width의 상관계수 계산하고 소수 둘째자리까지 출력하시오

In [19]:
import pandas as pd
from sklearn.datasets import load_iris

# iris 데이터셋 로드
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)

import scipy.stats
# for i in dir(pd.DataFrame):
#     if 'cor' in i:
#         print(i)
# help(pd.DataFrame.corr)

round(df[['sepal length (cm)','sepal width (cm)']].corr().iloc[0,1],2)

-0.12